In [108]:
import pandas as pd
import harboropt_lp_valueNewMethod
from ortools.linear_solver import pywraplp
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from tqdm import tnrange
import seaborn as sns
%reload_ext autoreload
%autoreload 2

In [109]:
portfolio_timespan = 30
discount_rate = 0.03
cost = 1

In [110]:
health_cost_emissions_la = pd.read_csv('data/pollutant_health_impacts/COBRA_LADWPplants_healthCosts.csv')

In [111]:
social_cost_carbon_short_ton = 46.3


discount_rate_inds = health_cost_emissions_la['discount_rate'] == discount_rate
la_inds = health_cost_emissions_la['county'] == 'LA'
pm25_inds = health_cost_emissions_la['pollutant'] == 'PM2.5' 
so2_inds = health_cost_emissions_la['pollutant'] == 'SO2'
nox_inds = health_cost_emissions_la['pollutant'] == 'NOx'

pm25_cost_short_ton_la = health_cost_emissions_la[discount_rate_inds & la_inds & pm25_inds]['US_HIGH_annual ($/ton)'].iloc[0]*-1

so2_cost_short_ton_la = health_cost_emissions_la[discount_rate_inds & la_inds & so2_inds]['US_HIGH_annual ($/ton)'].iloc[0]*-1

nox_cost_short_ton_la = health_cost_emissions_la[discount_rate_inds & la_inds & nox_inds]['US_HIGH_annual ($/ton)'].iloc[0]*-1


In [112]:
growth_rate = 1.0 + discount_rate

value_decay_1 = pow(growth_rate, -portfolio_timespan)
value_decay_2 = pow(growth_rate, -1)
try:
    extrapolate = cost * (1.0 - value_decay_1) / (1.0-value_decay_2)
except ZeroDivisionError:
    extrapolate = cost

In [113]:
resources = pd.read_csv('data/resources.csv')
resources = resources.set_index('resource')
resource_disp = resources[resources['dispatchable']=='y']
resource_nondisp = resources[resources['dispatchable']=='n']


resource_costs = pd.read_csv('data/resource_projected_costs_ee_utilitycosts.csv')
resource_costs = resource_costs[resource_costs['cost_decline_assumption']=='moderate']

In [114]:
#Set up Harbor demand profile.
demand = pd.read_csv('data/harbor_hourly_gen_emissions_2019.csv')
demand = demand.fillna(0)
annual_demand = demand['load_mw'].sum()
peak_demand = demand['load_mw'].max()

In [115]:
profiles = pd.read_csv('data/gen_profiles.csv')

In [117]:
#Write a csv of LCOE for each resource in each build year.
lcoe_per_mwh_by_resource_df = pd.DataFrame()
lcoe_per_mwh_w_emissions_by_resource_df = pd.DataFrame()

lcoe_per_mwh_by_resource = {}

for i, resource in enumerate(resources.index):

    lcoe_per_mwh_by_resource_df.loc[i,'resource']=resource
    lcoe_per_mwh_w_emissions_by_resource_df.loc[i,'resource']=resource

    lcoe_per_mwh_by_resource[resource]={}
    
    resource_inds = resource_costs['resource']==resource
    
    for year in range(16):
        
        discount_factor = pow(growth_rate, -(year))
        print(year, discount_factor)
        
        lcoe_per_mwh_by_resource[resource][year]={}

        cost_year = 2020+year
        print(cost_year)

        capex_inds = resource_costs['cost_type']=='capex_per_kw'
        fixed_inds = resource_costs['cost_type']=='fixed_per_kw_year'
        variable_inds = resource_costs['cost_type']=='variable_per_mwh'

        capex = resource_costs.loc[capex_inds & resource_inds, str(cost_year)].item() * 1000 * discount_factor
        if 'Ventilation' in resource:
            print(resource, cost_year, capex)

        fixed = resource_costs.loc[fixed_inds & resource_inds, str(cost_year)].item() * 1000
        fixed_extrapolated = fixed * extrapolate * discount_factor
        if 'Ventilation' in resource:
            print(resource, fixed_extrapolated)

        #For dispatchable resources, calculate extrapolated variable cost and annual generation per mw of capacity.
        if resource in resource_disp.index:

            resource_monetized_emissions_mwh = (resource_disp.loc[resource, 'co2_short_tons_per_mwh']*social_cost_carbon_short_ton) + resource_disp.loc[resource, 'nox_lbs_per_mwh']/2000*nox_cost_short_ton_la + resource_disp.loc[resource, 'so2_lbs_per_mwh']/2000*so2_cost_short_ton_la + resource_disp.loc[resource, 'pm25_lbs_per_mwh']/2000*pm25_cost_short_ton_la

            #Calculated extrapolated variable cost and add to lcoe dictionary. Excludes health impacts of monetized emissions.
            if 'gas' in resource:

                variable_om_inds = resource_costs['cost_type']=='variable_per_mwh'
                variable_fuel_cost_inds = resource_costs['cost_type']=='fuel_costs_per_mmbtu'
                heat_rate_inds = resource_costs['cost_type']=='heat_rate_mmbtu_per_mwh'

                variable_om_cost = resource_costs.loc[variable_om_inds & resource_inds,str(cost_year)].item() 
                variable_fuel_cost = resource_costs.loc[variable_fuel_cost_inds & resource_inds, str(cost_year)].item()
                variable_heat_rate = resource_costs.loc[heat_rate_inds & resource_inds, str(cost_year)].item()

                variable_cost = variable_om_cost + (variable_fuel_cost*variable_heat_rate)

                variable_cost_w_cobenefits = variable_om_cost + (variable_fuel_cost*variable_heat_rate) + resource_monetized_emissions_mwh


            else:
                variable_cost_inds = resource_costs['cost_type']=='variable_per_mwh'
                variable_cost = resource_costs.loc[variable_cost_inds & resource_inds,str(cost_year)].item()

                variable_cost_w_cobenefits = variable_cost + resource_monetized_emissions_mwh

            variable_cost_extrapolated = variable_cost * extrapolate * discount_factor
            if 'Ventilation' in resource:
                print(resource, 'variable_cost_extrapolated=', variable_cost_extrapolated)
            variable_cost_w_cobenefits_extrapolated = variable_cost_w_cobenefits * extrapolate * discount_factor
            if 'Ventilation' in resource:
                print(resource, 'variable_cost_w_cobenefits_extrapolated=', variable_cost_w_cobenefits_extrapolated)

            mwh_per_mw = annual_demand / peak_demand
            if 'Ventilation' in resource:
                print(resource, 'mwh_per_mw', mwh_per_mw)

        else:

            profile_max = max(profiles[resource])
            summed_gen = sum(profiles[resource] / profile_max)
            
            if 'Ventilation' in resource:
                print(resource, 'profile_max=', profile_max)
            if 'Ventilation' in resource:
                print(resource, 'summed_gen=', summed_gen)
            
            variable_cost_inds = resource_costs['cost_type']=='variable_per_mwh'
            variable_cost = resource_costs.loc[variable_cost_inds & resource_inds, str(cost_year)].item()
            variable_cost_extrapolated = variable_cost * extrapolate * discount_factor
            if 'Ventilation' in resource:
                print(resource, 'variable_cost_extrapolated', variable_cost_extrapolated)
            
            mwh_per_mw = summed_gen
            if 'Ventilation' in resource:
                print(resource, 'mwh_per_mw', mwh_per_mw)

            
        #Calculate lcoe_per_mwh without co-benefits.
        variable_costs = variable_cost_extrapolated * mwh_per_mw
        
        lcoe_per_mw = capex + fixed_extrapolated + variable_costs
        
        lcoe_per_mwh = lcoe_per_mw / (mwh_per_mw*portfolio_timespan)
        if 'Ventilation' in resource:
            print(resource, 'lcoe', lcoe_per_mwh)

        lcoe_per_mwh_by_resource[resource][year] = lcoe_per_mwh
        
        lcoe_resource_inds = lcoe_per_mwh_by_resource_df['resource']==resource
        lcoe_per_mwh_by_resource_df.loc[lcoe_resource_inds, cost_year] = lcoe_per_mwh

        #Calculate lcoe_per_mwh with emissions.
        if 'gas' in resource:
            lcoe_resource_inds = lcoe_per_mwh_by_resource_df['resource']==resource
            variable_costs_w_cobenefits = variable_cost_w_cobenefits_extrapolated * mwh_per_mw
            lcoe_per_mw_w_cobenefits = capex + fixed_extrapolated + variable_costs_w_cobenefits
            lcoe_per_mwh_w_cobenefits = lcoe_per_mw_w_cobenefits / (mwh_per_mw*portfolio_timespan)

            lcoe_per_mwh_w_emissions_by_resource_df.loc[lcoe_resource_inds, cost_year] = lcoe_per_mwh_w_cobenefits

lcoe_per_mwh_by_resource_df['lcoe_type'] = 'no_emissions'
lcoe_per_mwh_w_emissions_by_resource_df['lcoe_type'] = 'w_resource_emissions'
lcoe_per_mwh_2020_2030 = pd.concat([lcoe_per_mwh_by_resource_df,lcoe_per_mwh_w_emissions_by_resource_df], ignore_index=True)

lcoe_per_mwh_2020_2030 = lcoe_per_mwh_2020_2030.dropna()
lcoe_per_mwh_2020_2030

0 1.0
2020
1 0.970873786407767
2021
2 0.9425959091337544
2022
3 0.9151416593531595
2023
4 0.8884870479156888
2024
5 0.8626087843841639
2025
6 0.8374842566836542
2026
7 0.8130915113433536
2027
8 0.7894092343139355
2028
9 0.7664167323436267
2029
10 0.7440939148967249
2030
11 0.722421276598762
2031
12 0.7013798801929728
2032
13 0.6809513399931775
2033
14 0.6611178058186189
2034
15 0.6418619473967174
2035
0 1.0
2020
1 0.970873786407767
2021
2 0.9425959091337544
2022
3 0.9151416593531595
2023
4 0.8884870479156888
2024
5 0.8626087843841639
2025
6 0.8374842566836542
2026
7 0.8130915113433536
2027
8 0.7894092343139355
2028
9 0.7664167323436267
2029
10 0.7440939148967249
2030
11 0.722421276598762
2031
12 0.7013798801929728
2032
13 0.6809513399931775
2033
14 0.6611178058186189
2034
15 0.6418619473967174
2035
0 1.0
2020
1 0.970873786407767
2021
2 0.9425959091337544
2022
3 0.9151416593531595
2023
4 0.8884870479156888
2024
5 0.8626087843841639
2025
6 0.8374842566836542
2026
7 0.8130915113433536
202

,resource,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,lcoe_type
0,gas_harbor,73.119636,72.148999,70.008556,68.430752,67.389544,67.141395,66.443495,65.276500,63.743905,61.827922,59.649360,57.739886,56.185220,54.875853,53.532839,51.949286,no_emissions
1,gas_ct,164.358556,160.039636,154.814176,149.489925,144.139299,140.268492,136.596514,132.570362,128.575594,124.480627,120.122158,116.138145,112.519023,109.142734,105.916857,102.621348,no_emissions
2,ci_shed_nonwatersystem,284.698009,276.405834,268.355179,260.539008,252.950494,245.583003,238.430100,231.485534,224.743237,218.197318,211.842056,205.671899,199.681455,193.865491,188.218923,182.736819,no_emissions
3,utility_solar_outofbasin,28.375434,26.496381,24.702715,22.991056,21.358150,19.800860,18.316167,16.901163,15.553049,14.269127,13.046803,12.554280,12.079378,11.621491,11.180029,10.754425,no_emissions
4,solar_rooftop_ci,33.812123,31.499392,29.292700,27.187831,25.180726,23.267474,21.444313,19.707617,18.053895,16.479788,14.982061,14.376360,13.793232,13.231877,12.691521,12.171417,no_emissions
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,FCZ7.Residential.SINGLEFAMILY.Refrigerator,44.333333,43.042071,41.788419,40.876327,39.685755,38.529859,37.407630,36.318088,35.260279,34.488753,33.236195,32.268150,32.263474,30.415827,29.529929,28.669834,no_emissions
109,FCZ7.Residential.SINGLEFAMILY.ResLightingEff,47.333333,45.954693,44.616206,43.011658,38.501105,60.095079,58.903059,66.402473,65.784103,57.225783,45.637760,33.953800,28.288988,27.692021,26.885457,26.316340,no_emissions
110,FCZ7.Residential.SINGLEFAMILY.WaterHeating,44.666667,43.365696,42.102617,41.486422,41.166567,40.542613,39.920083,40.654576,39.996735,39.342726,38.444852,37.565906,37.173134,36.317405,35.259616,34.446591,no_emissions
111,gas_harbor,142.717295,139.719541,135.611023,132.122468,129.226162,127.176946,124.730438,121.865765,118.684939,115.168731,111.436554,108.018715,104.999617,102.268472,99.545090,96.621374,w_resource_emissions


In [123]:
lcoe_per_mwh_2020_2030.to_csv('data/lcoe_per_mwh_2020_2035_1_31_22.csv')

#### Concatenate LCOE files for 2020-2035.

In [ ]:
lcoe_2020_2024 = pd.read_csv('model_run_results/lcoe_per_mwh_2024.csv')
lcoe_2025_2030 = pd.read_csv('model_run_results/lcoe_per_mwh_2030.csv')
lcoe_2031_2035 = pd.read_csv('model_run_results/lcoe_per_mwh_2035.csv')

lcoe_2020_2030 = lcoe_2020_2024.merge(lcoe_2025_2030, how='outer', on=['resource','lcoe_type'])
lcoe_2020_2035 = lcoe_2020_2030.merge(lcoe_2031_2035, how='outer', on=['resource','lcoe_type'])

lcoe_2020_2035 = lcoe_2020_2035[['resource', 'lcoe_type', '2020','2021', '2022', '2023',
       '2024', '2025', '2026', '2027', '2028', '2029', '2030',
        '2031', '2032', '2033', '2034', '2035']]

disp_resources = ['gas_harbor','gas_ct','ci_shed_nonwatersystem']
lcoe_2020_2035 = lcoe_2020_2035[lcoe_2020_2035['resource'].isin(disp_resources)==False]

lcoe_gas_harbor = pd.read_csv('model_run_results/lcoe_per_mwh_gasHarbor_2035.csv')
lcoe_gas_ct = pd.read_csv('model_run_results/lcoe_per_mwh_gasCT_2035.csv')
lcoe = pd.concat([lcoe_2020_2035,lcoe_gas_harbor], ignore_index=True)
lcoe = pd.concat([lcoe,lcoe_gas_ct], ignore_index=True)
lcoe['units']='2018$/mwh'

In [ ]:
lcoe.to_csv('model_run_results/lcoe_2020_2035.csv')